In [2]:
!pip install pulp

In [4]:
import numpy as np
from scipy.sparse import csr_matrix
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, value

# Define the knowledge graph
knowledge_graph = {
    ('ProductA', 'Completed'): 10,
    ('ProductA', 'InProgress'): 5,
    ('ProductA', 'Pending'): 2,
    ('ProductB', 'Completed'): 8,
    ('ProductB', 'InProgress'): 7,
    ('ProductB', 'Pending'): 3,
    ('ProductC', 'Completed'): 15,
    ('ProductC', 'InProgress'): 4,
    ('ProductC', 'Pending'): 1
}

# Extract unique nodes (products) and statuses
products = list(set([edge[0] for edge in knowledge_graph.keys()]))
statuses = list(set([edge[1] for edge in knowledge_graph.keys()]))

# Create a mapping between node names and indices
product_indices = {product: i for i, product in enumerate(products)}
status_indices = {status: i for i, status in enumerate(statuses)}

# Create a matrix of the knowledge graph
matrix = csr_matrix((np.ones(len(knowledge_graph)), ([product_indices[edge[0]] for edge in knowledge_graph.keys()],
                                                     [status_indices[edge[1]] for edge in knowledge_graph.keys()])),
                    shape=(len(products), len(statuses)))

# Create a vector of the weights of the edges in the knowledge graph
weights = np.array([knowledge_graph[edge] for edge in knowledge_graph.keys()])

# Create a vector of the variables
variables = [LpVariable(f'{product}_{status}', lowBound=0) for product in products for status in statuses]

# Define the objective function (minimizing a dummy objective as we are interested in constraints)
objective = lpSum([0 * var for var in variables])  # Dummy objective

# Create the constraints
constraints = []

# Add constraint: sum of all products' statuses should equal the known weights
for i, (product, status) in enumerate(knowledge_graph.keys()):
    constraints.append(variables[i] == weights[i])

# Create the model
model = LpProblem("Product Status Query", LpMinimize)

# Add the objective function and constraints to the model
model += objective
for c in constraints:
    model += c

# Solve the model
model.solve()

# Function to query the number of products based on the specified status
def query_products(query):
    query = query.strip().lower()
    if "how many" in query and "completed" in query:
        query_status = "Completed"
    elif "how many" in query and "in progress" in query:
        query_status = "InProgress"
    elif "how many" in query and "pending" in query:
        query_status = "Pending"
    else:
        raise ValueError("Query not understood. Please ask about 'completed', 'in progress', or 'pending' products.")

    relevant_vars = [var for var in variables if query_status in var.name]
    total = sum([value(var) for var in relevant_vars])
    return total

# Ask the user for a query
user_query = input("Please enter your query about the products: ")

# Query the number of products based on the user-defined query
try:
    total_products = query_products(user_query)
    print(f"Number of products that are {user_query.split()[-1].lower()}: {total_products}")
except ValueError as e:
    print(e)

# Generate the linear equation (not strictly necessary for the query, but included for completeness)
equation = ""
for i, var in enumerate(variables):
    if value(var) > 0:
        equation += f"{value(var)} * {var.name} + "
equation = equation.rstrip(" + ") + " = Total"
print("Linear equation:", equation)


Please enter your query about the products: how many products completed
Number of products that are completed: 16.0
Linear equation: 10.0 * ProductB_Pending + 5.0 * ProductB_Completed + 2.0 * ProductB_InProgress + 8.0 * ProductC_Pending + 7.0 * ProductC_Completed + 3.0 * ProductC_InProgress + 15.0 * ProductA_Pending + 4.0 * ProductA_Completed + 1.0 * ProductA_InProgress = Total
